**Data Creation for GLP Argentinian Sector**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import requests
import numpy as np
from sqlalchemy import create_engine
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#Path with location of the files

path = '/content/drive/MyDrive/Cursos 2022/SQL CH/Trabajo_practico/Data'

In [4]:
#Read dataset related to Fractionators

fracc = pd.read_csv(path + '/comercializacin-hidrocarburos-glp-plantas-fraccionadoras.csv', decimal=',')

fracc = fracc.drop(['nombre_planta','operador','departamento','geojson', 'geom'], axis=1)

fracc = fracc.drop_duplicates(['direccion', 'localidad', 'provincia'])[:5]

fracc = fracc.rename(columns={'razon_social':'Razon_social', 'propietario':'CUIL', 'direccion':'Direccion', 'localidad':'Localidad', 'provincia':'Provincia'})

fracc['Actividad_principal'] = 'Fraccionadora'

# Tables Creation

In [5]:
#Read dataset related to Distribuitors

distr = pd.read_csv(path + '/distribuidoras-de-glp.csv', decimal=',')

distr = distr.drop(['actividad','partido','geojson'], axis=1)

distr = distr.drop_duplicates(['direccion', 'localidad', 'provincia'])

distr = distr.rename(columns={'empresa':'Razon_social', 'cuit':'CUIL', 'direccion':'Direccion', 'localidad':'Localidad', 'provincia':'Provincia'})

distr['Actividad_principal'] = 'Distribuidora'

In [6]:
#Read dataset related to Producers

prod = pd.read_csv(path + '/productores-de-glp.csv', decimal=',')

prod = prod.drop(['nombre_boca','operador', 'departamento','geojson', 'geom'], axis=1)

prod = prod.drop_duplicates(['direccion', 'localidad', 'provincia'])

prod = prod.rename(columns={'razon_social':'Razon_social', 'propietario':'CUIL', 'direccion':'Direccion', 'localidad':'Localidad', 'provincia':'Provincia'})

prod['Actividad_principal'] = 'Productor'

In [7]:
#We concat all the results relates to companies

empr = pd.concat([fracc, distr, prod])

#We check that there are no records with the same, CUIL, Principal activity and Direction. These guarantees that it might exist a company with the same name, CUIL, principal activity but with different address

empr[empr.duplicated(subset=['CUIL', 'Actividad_principal', 'Direccion'], keep='first')]




,Razon_social,CUIL,Direccion,Localidad,Provincia,Actividad_principal


In [10]:
empr = empr.drop_duplicates('CUIL')

empr['id_empresa'] = range(1, len(empr)+1)

In [11]:
#Table Related to all the possible catogories

act_principal = {'Fraccionadora', 'Distribuidora', 'Productor'}

cat = pd.DataFrame(act_principal, columns=['Actividad_principal'])

cat['id_categoria'] = range(1,4)
cat = cat[['id_categoria', 'Actividad_principal']]

In [12]:
#Alocation of company and category

_empr_cat = empr.join(cat.set_index('Actividad_principal'), on='Actividad_principal')

_empr_cat = _empr_cat[['id_empresa', 'id_categoria']]


In [16]:
#Table with locations

loc = empr[['Localidad', 'Provincia']].drop_duplicates()
loc['id_localidad'] = range(1, len(loc)+1)
loc = loc[['id_localidad','Localidad', 'Provincia']]
loc[:3]

,id_localidad,Localidad,Provincia
0,1,CHIVILCOY,BUENOS AIRES
0,2,ELDORADO,MISIONES
1,3,RECONQUISTA,SANTA FE


In [24]:
empresas = empr.join(loc.set_index(['Localidad', 'Provincia']), on=['Localidad', 'Provincia'])
empresas = empresas[['id_empresa','Razon_social',	'CUIL', 'Localidad',	'Direccion', 'id_localidad']]
empresas.dtypes



id_empresa       int64
Razon_social    object
CUIL             int64
Localidad       object
Direccion       object
id_localidad     int64
dtype: object

In [18]:
#Read dataset related to Sells

vtas = pd.read_csv(path + '/ventas-de-productores-de-glp.csv', decimal=',')

vtas = vtas.drop(['fecha_data','Actividad Comprador', 'Volumen Fuera','Precio Fuera Iva', 'Precio Fuera Sin Iva', 'Volumen Res 844/2007', 'Precio Res 844/2007 Iva', 'Precio Res 844/2007 Sin Iva'], axis=1)

vtas = vtas.rename(columns={ 'Vendedor':'Empresa_vendedora', 'Comprador':'Empresa_compradora','Planta Carga':'Planta_carga', 'Tipo Producto':'Tipo_producto', 'Volumen Dentro':'Volumen', 'Precio Dentro Iva':'Precio_con_iva', 'Precio Dentro Sin Iva':'Precio_sin_iva'})
#'Cuit Vendedor':'id_empresa_venta' 'Cuit Comprador':'id_empresa_compra',


In [19]:
ventas = vtas[['Año', 'Mes',	'Cuit Vendedor',	'Empresa_vendedora',	'Cuit Comprador',	'Empresa_compradora', 'Planta_carga', 'Volumen', 'Tipo_producto']]

ventas['id_venta'] = range(1, len(vtas)+1)

ventas['Cuit Comprador'] = ventas['Cuit Comprador'].replace('-', '', regex=True).astype(int)


#ventas = ventas.join(empresas, on='CUIL', how='left')

ventas = pd.merge(ventas, empresas, how='inner', left_on='Cuit Comprador', right_on='CUIL').drop(['Empresa_compradora', 'Cuit Comprador','Razon_social', 'CUIL', 'Localidad', 'Direccion', 'id_localidad'], axis=1).rename(columns={'id_empresa':'id_empresa_compra'})

ventas = pd.merge(ventas, empresas, how='inner', left_on='Cuit Vendedor', right_on='CUIL').drop(['Empresa_vendedora', 'Cuit Vendedor','Razon_social', 'CUIL', 'Localidad', 'Direccion', 'id_localidad'], axis=1).rename(columns={'id_empresa':'id_empresa_venta'}) #

#Set the categories of each company

ventas = pd.merge(ventas, _empr_cat, how='inner', left_on='id_empresa_compra', right_on='id_empresa').drop(['id_empresa'], axis=1).rename(columns={'id_categoria':'id_categoria_compra'})

ventas = pd.merge(ventas, _empr_cat, how='inner', left_on='id_empresa_venta', right_on='id_empresa').drop(['id_empresa'],  axis=1).rename(columns={'id_categoria':'id_categoria_venta'})


#.set_index('Cuit Vendedor')

df_ventas = ventas[['id_venta', 'Año',	'Mes', 'id_empresa_compra',	'id_empresa_venta',	'id_categoria_compra',	'id_categoria_venta']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
df_prod = vtas[['Tipo_producto', 'Precio_sin_iva', 'Precio_con_iva']]
df_prod['Precio_sin_iva'] = pd.to_numeric(df_prod['Precio_sin_iva'])
df_prod['Precio_con_iva'] = pd.to_numeric(df_prod['Precio_con_iva'])

df_prod = df_prod[df_prod['Precio_con_iva'] > 0]
df_prod.drop_duplicates(subset=['Tipo_producto'], inplace=True)

df_prod['id_producto'] = range(1, len(df_prod)+1)

df_prod = df_prod[['id_producto', 'Tipo_producto', 'Precio_sin_iva', 'Precio_con_iva']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
df_detalle = ventas[['Planta_carga', 'Volumen', 'Tipo_producto', 'id_venta']]
df_detalle['Volumen'] = pd.to_numeric(df_detalle['Volumen'])

df_detalle['id_detalle_venta'] = range(1, len(df_detalle)+1)

df_detalle = df_detalle.join(df_prod.set_index('Tipo_producto'), on = 'Tipo_producto').drop(['Precio_con_iva', 'Precio_sin_iva'], axis=1)

df_detalle = df_detalle[['id_detalle_venta','Planta_carga', 'Volumen', 'id_venta', 'id_producto']]

df_detalle

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id_detalle_venta,Planta_carga,Volumen,id_venta,id_producto
0,1,REF BAHIA BLANCA |Oper:Petrobras Argentina S.A.|,182.120,1,1.0
1,2,REF BAHIA BLANCA |Oper:Petrobras Argentina S.A.|,184.360,287,1.0
2,3,REF BAHIA BLANCA |Oper:Petrobras Argentina S.A.|,92.420,583,1.0
3,4,REF BAHIA BLANCA |Oper:Petrobras Argentina S.A.|,24.262,872,1.0
4,5,REF BAHIA BLANCA |Oper:Petrobras Argentina S.A.|,227.940,1169,1.0
...,...,...,...,...,...
12190,12191,SAN SEBASTIAN |Oper:Petrolera LF Company SRL|,0.000,5571,2.0
12191,12192,SAN SEBASTIAN |Oper:Petrolera LF Company SRL|,0.000,6321,2.0
12192,12193,SAN SEBASTIAN |Oper:Petrolera LF Company SRL|,0.000,7010,2.0
12193,12194,SAN SEBASTIAN |Oper:Petrolera LF Company SRL|,0.000,7328,2.0


In [ ]:
#Read dataset related to Sells

vtas_dis = pd.read_csv(path + '/ventas-de-distribuidor-envases-de-glp.csv', decimal=',')


#vtas_dis = vtas.drop(['fecha_data','Actividad Comprador', 'Volumen Fuera','Precio Fuera Iva', 'Precio Fuera Sin Iva', 'Volumen Res 844/2007', 'Precio Res 844/2007 Iva', 'Precio Res 844/2007 Sin Iva'], axis=1)

#vtas_dis = vtas.rename(columns={'Cuit Vendedor':'id_empresa_venta', 'Vendedor':'Empresa_vendedora','Cuit Comprador':'id_empresa_compra', 'Comprador':'Empresa_compradora','Planta Carga':'Planta_carga', 'Tipo Producto	':'Tipo_producto', 'Volumen Dentro':'Volumen', 'Precio Dentro Iva':'Precio_con_iva', 'Precio Dentro Sin Iva':'Precio_sin_iva'})


In [27]:
#Tables to CSV

ruta = '/content/drive/MyDrive/Cursos 2022/SQL CH/Trabajo_practico/Data/Output'

#Empresa
empresas.to_csv(ruta + '/Empresa.csv', encoding="latin1", index=False)

#Empresa_categoria
cat.to_csv(ruta + '/Categoria.csv', encoding="latin1", index=False)

#Localidad
loc.to_csv(ruta + '/Localidad.csv', encoding="latin1", index=False)

#Ventas
df_ventas.to_csv(ruta + '/Venta.csv', encoding="latin1", index=False)

#Producto
df_prod.to_csv(ruta + '/Producto.csv', encoding="latin1", index=False)

#Detalle_venta
df_detalle.to_csv(ruta + '/Detalle_venta.csv', encoding="latin1", index=False)

#Empresa_categoria
_empr_cat.to_csv(ruta + '/Empresa_categoria.csv', encoding="latin1", index=False)



In [ ]:
empr

,Razon_social,CUIL,Direccion,Localidad,Provincia,Actividad_principal
0,AMARILLA GAS S.A,30515637210,RUTA NACIONAL 5 N/KM 157,CHIVILCOY,BUENOS AIRES,Fraccionadora
1,AMARILLA GAS S.A,30515637210,RUTA PROVINCIAL Nº 23 Y PEDRIEL N/KM,MORENO,BUENOS AIRES,Fraccionadora
2,AMARILLA GAS S.A,30515637210,PRIMERA JUNTA Y CASEROS N/KM,TANDIL,BUENOS AIRES,Fraccionadora
3,AMARILLA GAS S.A,30515637210,RUTA PROVINCIAL Nº 68 N/KM 174,CERRILLOS,SALTA,Fraccionadora
4,AMARILLA GAS S.A,30515637210,RUTA NACIONAL 11 N/KM 999,RESISTENCIA,CHACO,Fraccionadora
...,...,...,...,...,...,...
37,YPF SOCIEDAD ANONIMA,30546689979,BARADERO N/KM S/N,LA PLATA,BUENOS AIRES,Productor
38,PETROBRAS ARGENTINA S.A.,30504077078,RUTA NACIONAL 3 N/KM 2620 - PLANTA CHIMEN AIKE-,RIO GALLEGOS,SANTA CRUZ,Productor
39,TOTAL AUSTRAL SOCIEDAD ANONIMA (SUCURSAL ARGEN...,30569719344,YACIMIENTO AGUADA PICHANA N/KM,AÑELO,NEUQUÉN,Productor
40,TOTAL AUSTRAL SOCIEDAD ANONIMA (SUCURSAL ARGEN...,30569719344,YACIMIENTO SAN ROQUE N/KM,AÑELO,NEUQUÉN,Productor
